# Single-use-seal

This notebook is about demonstrating how Peter Todd's [single-use-seal](https://petertodd.org/2016/commitments-and-single-use-seals) is supposed to work, and how it applies to bitcoin transactions

In [1]:
import ecc.ecc as ecc
import ecc.util as util
from os import urandom
from random import randrange
import hashlib, hmac

## Announcement of a seal

The point of a single-use-seal is that we can make it known before the data it commits to even exists.

In [2]:
# let's take a public key. This will be our seal.
seal = ecc.PrivateKey(int(b"a8c2ddfc451aeded83faf5030dc6c726c0c2f1f0bc91afbd2e82ca75a9c176de", 16))
print(f"seal is {seal.point}")

seal is 03e6d191c3c3a6313e0157e2dd9b5c53aa6d58ce3d6c6dc4266274bd0838a80420


### Defining the commitment
The commitment can simply be the hash of a message we want to commit to.

In [3]:
# let's hash an arbitrary string
msg = b"This is my message"

commitment = hashlib.sha256(msg).digest()
print(f"commitment is {commitment.hex()}")

commitment is 3311b7c0bd91b6c73a38212de8ade31c51910f17480ad212ed2b9798a35b7747


### Closing the seal over the commitment

In [4]:
# now we can sign the commitment with the private key that fit our seal
signature = seal.sign(commitment)
print(f"Signature is {signature}")

Signature is 3045022100dccc12fbdc67b075eb35dfd723ddc004e43648b9d1fe1aa46a3805430cc3556c0220018b8a689fcd8ace6f681489efe61da47d2e2e6bc4153ed816074f473461bc2a


In [5]:
# and anyone with the message, the seal and the signature can verify it.
print(f"Signature verify for public key {seal.point.sec().hex()}: {seal.point.verify(commitment, signature)}")

Signature verify for public key 03e6d191c3c3a6313e0157e2dd9b5c53aa6d58ce3d6c6dc4266274bd0838a80420: True


In [6]:
# I can link multiple seals together committing another seal inside the previous one
next_seal = ecc.PrivateKey(int(b"378ee5ef6c5e466b295b4f83716c94689086390d307090cb800300acfaba0706", 16))
msg = f"This is my message\nnext seal:{next_seal.point}"
commitment = hashlib.sha256(bytes(msg, 'utf-8')).digest()
print(f"commitment is {commitment.hex()}")

commitment is 83c65a4fbb191322d6457c23315c42b1ef07c8f18f6d2156c69e55d452d88a5f


In [7]:
# Now I sign the commitment and thereby I also commit to the next seal
signature = seal.sign(commitment)
print(f"Signature verify for public key {seal.point.sec().hex()}: {seal.point.verify(commitment, signature)}")

Signature verify for public key 03e6d191c3c3a6313e0157e2dd9b5c53aa6d58ce3d6c6dc4266274bd0838a80420: True


## Problem
I can close the seal on multiple messages, making them, well, not that much "single-use". That's because I can sign any message with the key and produce valid signatures.
## Solution
Use bitcoin UTXOs as seal, because they are provably only spendable once (or Bitcoin would be broken)

In [8]:
class Seal:
    def __init__(self, wif: str, prevtx: str, vout: int):
        self.keys = ecc.PrivateKey.from_wif(wif)
        self.address = self.keys.point.address()
        self.prevtx = bytes.fromhex(prevtx)[::-1]
        self.vout = vout
        
utxo0 = "fa0b91b1148789e8eb05cd638de7a04dbb8774eada728c4dacc8d3a03100cbcf"
wif0 = "cP9gNepnEeDrZzD489L9YRhaidDC8Kwctr3gLCwRZfgveCJgVJwS"

utxo1 = "2d0be74700f7551991bc06f285006d466abe27ba78d1b769b913d7caeb9f2f3a"
wif1 = "cNvNezudSk1WHdU1tqoxXHFjbMLy7kfywqBUHeW4jpPKk2zCZ5jn"

# let's use this UTXO as a first seal
genesis_seal = Seal(wif0, utxo0, 0)
print(genesis_seal.address)
# I can create my commitment to the message and to the next seal (a completely unrelated bitcoin transaction)
next_seal = Seal(wif1, utxo1, 0)
print(next_seal.address)
msg = f"This is my message\nnext seal:{next_seal.prevtx[::-1].hex()}:{next_seal.vout}"
print(msg)
commitment = hashlib.sha256(bytes(msg, 'utf-8')).digest()

# And now sign and eventually spend the transaction that serves as genesis_seal to close the commitment
signature = genesis_seal.keys.sign(commitment)
print(f"Signature verify for public key {genesis_seal.keys.point}: {genesis_seal.keys.point.verify(commitment, signature)}")


mn4sNZMuJnBFTbWYcn1FZ3inBL1Ss3Sjny
mr9awu69wnCd3AM6iEEeDqSuSZoYuNq6Sk
This is my message
next seal:2d0be74700f7551991bc06f285006d466abe27ba78d1b769b913d7caeb9f2f3a:0
Signature verify for public key 03c6999e2eec6c9e2e644a2f585df1e5ebf52539433ca8f7ba879dad6608896205: True


## Let's do it with real transactions

In [9]:
import tx.tx as tx
import tx.script as script
from io import BytesIO

In [10]:
vin0 = tx.TxIn(genesis_seal.prevtx, genesis_seal.vout)

# we need one standard output that pay to any address, we don't care
some_pubkey = "0339068b1a4e301ffe0ba7ce40e9ce0b7a58a8ba19977d621820b93286015fa69d"
scriptpubkey = script.p2pkh_script(util.hash160(bytes.fromhex(some_pubkey)))

# we enbed the commitment in an op_return (old-school)
opreturn = script.opreturn_script(commitment)

vout0 = tx.TxOut(99900000, scriptpubkey)
vout1 = tx.TxOut(0, opreturn)
tx0 = tx.Tx(2, [vin0], [vout0,vout1], 0)
print(tx0)

tx: f26589047282d8dac8ac634b777efa2fabea8b30da0ea9348225234906f6f4c4
version: 2
tx_ins:
fa0b91b1148789e8eb05cd638de7a04dbb8774eada728c4dacc8d3a03100cbcf:0
tx_outs:
99900000:OP_DUP OP_HASH160 d5c014cc5f6aaff81278c304370de4abe61260f7 OP_EQUALVERIFY OP_CHECKSIG
0:OP_RETURN 718a208aca8a1b7ee7e50a014868a417edd41f17c77912ce1ca80309af53a97f
locktime: 0


In [11]:
# to keep it simple, let's just manually copy/paste the scriptpubkey of the UTXO we're spending
prev_scriptpubkey = script.Script.parse(BytesIO(bytes.fromhex("1976a91447d9f4dec0b895b50640d7f269251f4bd17b6a4e88ac")))

tx0.sign_input(0, genesis_seal.keys, prev_scriptpubkey)
tx0.serialize().hex()

'0200000001cfcb0031a0d3c8ac4d8c72daea7487bb4da0e78d63cd05ebe8898714b1910bfa000000006b483045022100db959d05571a521014dd5abfa9b7be09ffb793249a383f37b4022fe400126314022059578f783933e028a2f1c9fffd287a08b4427feb856ce5bff9e27a22d094292f012103c6999e2eec6c9e2e644a2f585df1e5ebf52539433ca8f7ba879dad6608896205ffffffff02605af405000000001976a914d5c014cc5f6aaff81278c304370de4abe61260f788ac0000000000000000226a20718a208aca8a1b7ee7e50a014868a417edd41f17c77912ce1ca80309af53a97f00000000'

## Last step: let's actually tweak the public key

### We reuse our tweaking code from the other book

In [12]:
def format_msg(msg):
    # create the lnpbp1 msg
    prefix = hashlib.sha256(b"LNPBP1").digest()
    lnpbp1_msg = prefix + hashlib.sha256(b"tag").digest() + msg.encode('utf-8')
    return lnpbp1_msg

def create_commitment(pubkey, msg):
    # implementation of LNPBP 1 (single key only)
    lnpbp1_msg = format_msg(msg)
    # HMAC s and P to get the tweaking factor f
    hmac_msg = hmac.digest(pubkey.sec(False), lnpbp1_msg, hashlib.sha256)
    f = int.from_bytes(hmac_msg, 'big')
    # assert f < n
    try:
        assert f < ecc.N
    except:
        print("ERROR: tweak overflow secp256k1 order")
    # Compute a new PrivateKey with f as secret
    return ecc.PrivateKey(f)

def verify_commitment(original_pubkey, msg, commitment):
    candidate = create_commitment(original_pubkey, msg)
    return candidate.point + original_pubkey == commitment

In [13]:
# extract a pubkey from our wallet
wallet_pubkey = ecc.S256Point.parse(bytes.fromhex("02e425a3a0248fdb65c452af38552e867929fa08a80e6aaf3effc294fa2b44ce0a"))

# format the msg we want to commit to
msg = f"This is my message\nnext seal:{next_seal.prevtx[::-1].hex()}:{next_seal.vout}"
fmt_msg = format_msg(msg)
print(f"the message we commit to is {fmt_msg.hex()}")

# create the tweaking factor from our msg and this pubkey
tweaking_factor = create_commitment(wallet_pubkey, msg)
print(f"tweaking_factor pubkey is {tweaking_factor.wif()} and {tweaking_factor.point}")

# tweak our pubkey
tweaked_pubkey = tweaking_factor.point + wallet_pubkey
print(f"tweaked_pubkey is {tweaked_pubkey}")
try:
    assert verify_commitment(wallet_pubkey, msg, tweaked_pubkey) == True
except:
    print("Verification failed")
    

# we can now easily encode the tweaked pubkey in an address we can pay to
print(f"tweaked address is {tweaked_pubkey.address()}")

the message we commit to is f508f28efcc071526ca886c8e07c69d4954e2ec9fc52ab8cccd129110c0040af2a1073a6e67f0e5f09a5957c659503c690efe7272be8313df872556a9a684d8c54686973206973206d79206d6573736167650a6e657874207365616c3a326430626537343730306637353531393931626330366632383530303664343636616265323762613738643162373639623931336437636165623966326633613a30
tweaking_factor pubkey is cRbwJMKQvU7ndkPFVsoXEaGvqoXZw5HUHTDgc5rNt4de67rGhtUN and 022d292aecb74ad3e29fa27570b3a1e608c49b9e06d393c69a0e0ba299b21d5e26
tweaked_pubkey is 03573b4fa3e687c865327341e60fe0e6005ca7acf65683390469ee64dba55255ce
tweaked address is mk3u4cfA6VKjTS4hqMT8msjw1uUoDSeaV7


In [16]:
# we can now spend this by importing the tweaked private key
wallet_privkey = ecc.PrivateKey.from_wif("cSgvpCBcTZnTMPmcsbDRxJmkUzXYnnzjSQYWz3uftsN4JPD2BZx5")

tweaked_privkey = ecc.PrivateKey((tweaking_factor.secret.to_int() + wallet_privkey.secret.to_int()) % ecc.N)
print(tweaked_privkey.point.address())

mk3u4cfA6VKjTS4hqMT8msjw1uUoDSeaV7
